In [3]:
%matplotlib notebook
import cv2
import numpy as np
from localBlur.likematlab import im2col, EM_GMM
import matplotlib.pyplot as plt
from scipy import stats

### 이미지를 흑백으로 변환

In [4]:
img = cv2.imread('../image/out_of_focus0015.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.normalize(img.astype('double'), None, 0.0, 1.0, cv2.NORM_MINMAX)

In [5]:
patchsize = 11

### localBlurScore 부분

In [6]:
im_height, im_width = img.shape
offset = int((patchsize -1)/2)
x_start = offset+1; x_end = im_width-offset;
y_start = offset+1; y_end = im_height-offset;
datasize = (x_end-x_start+1)*(y_end-y_start+1)

### GradientHistogramSpan 부분

In [7]:
im_height, im_width = img.shape
offset = int((patchsize -1)/2)

In [8]:
dx = np.c_[np.diff(img),-img[:,-1]]
dy = np.r_['0,2',np.diff(img,axis=0),-img[-1,:]]

In [9]:
mag = np.sqrt(np.square(dx) + np.square(dy))

In [10]:
mag_col = im2col(mag,[patchsize,patchsize])
mag_col_dup = np.hstack((mag_col,-mag_col))

In [11]:
num = mag_col.shape[1]
sigma1 = np.zeros((1,num))

In [ ]:
for i in range(num):
    # time consuming 부분 
    [V1,V2] = EM_GMM(mag_col_dup[:,i])
    s0 = np.sqrt(V1)
    s1 = np.sqrt(V2)
    if s0 > s1 :
        temp = s0; s0 = s1; s1 = temp
    
    sigma1[0,i] = s1

C:\Users\sangjaekang123\Desktop\2017-1학기\컴퓨터 비전\project\blur_detection\python\localBlur\likematlab.py:48: RuntimeWarning: divide by zero encountered in true_divide
  return np.exp(-(XxX)/(2*V)) / np.sqrt(2*np.pi*V)
C:\Users\sangjaekang123\Desktop\2017-1학기\컴퓨터 비전\project\blur_detection\python\localBlur\likematlab.py:48: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(-(XxX)/(2*V)) / np.sqrt(2*np.pi*V)
C:\Users\sangjaekang123\Desktop\2017-1학기\컴퓨터 비전\project\blur_detection\python\localBlur\likematlab.py:48: RuntimeWarning: overflow encountered in true_divide
  return np.exp(-(XxX)/(2*V)) / np.sqrt(2*np.pi*V)


위의 연산에서 시간을 엄청 잡아먹음 수정이 필요

In [ ]:
q = np.reshape(sigma1, [im_width-patchsize+1,im_height-patchsize+1])
q = q[4:(im_width-patchsize+1-3),4:(im_height-patchsize+1-3)]
q = np.pad(q,offset+3,'edge')

In [ ]:
q = np.transpose(q)
plt.imshow(q)